In [30]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from shapely.geometry import LineString

import geopandas as gpd
import geopy

tqdm.pandas()
sns.set(style="whitegrid")
import warnings 
warnings.filterwarnings("ignore")

In [31]:
incidents = pd.read_csv('DATA.csv')

In [32]:
incidents.head(1)

,Folder name,Folder color,Latitude,Longitude,Title,Description,Color,Phone number,Timestamp,Pin icon code
0,NaN,71b300,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,NaN,2024-02-08T17:16:00Z,-1


In [33]:
incidents.tail(1)

,Folder name,Folder color,Latitude,Longitude,Title,Description,Color,Phone number,Timestamp,Pin icon code
122,NaN,71b300,37.089224,-8.298158,Setembro de 1986,Bombas Detonadas em Empreendimentos Turísitico...,3f51b5,NaN,2024-02-08T17:57:28Z,-1


In [34]:
incidents['ID'] = range(1, len(incidents) + 1)
incidents.set_index('ID', inplace = True)

In [35]:
incidents

,Folder name,Folder color,Latitude,Longitude,Title,Description,Color,Phone number,Timestamp,Pin icon code
ID,,,,,,,,,,
1,NaN,71b300,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,NaN,2024-02-08T17:16:00Z,-1
2,NaN,71b300,38.010202,-7.870127,1 de Fevereiro de 1985,"Bombas Detonadas, 8 dispositivos no Bairro Ale...",3f51b5,NaN,2024-02-08T17:37:17Z,-1
3,NaN,71b300,38.748041,-9.188370,1 de Junho de 1984,"Agressão, disparos contra administrador de uma...",ff5722,NaN,2024-02-08T17:10:19Z,-1
4,NaN,71b300,39.973942,-8.627772,1 de Maio de 1984,"Vandalismo, colocação de Pregos na Estrada Nac...",ffeb3b,NaN,2024-02-08T17:06:30Z,-1
5,NaN,71b300,38.815583,-9.223445,2 de Novembro de 1983,Assalto à dependência do Crédito Predial Portu...,71b300,NaN,2024-02-08T14:42:44Z,-1
...,...,...,...,...,...,...,...,...,...,...
119,NaN,71b300,38.768374,-9.294940,Janeiro de 1982,Bombas Detonadas no Posto da Guarda Nacional R...,3f51b5,NaN,2024-02-08T13:58:46Z,-1
120,NaN,71b300,39.694550,-8.130265,Julho de 1980,Constante Vandalismo contra veiculos da Políci...,ffeb3b,NaN,2024-02-08T12:23:55Z,-1
121,NaN,71b300,38.697433,-9.423230,Maio de 1981,Explosivos Detonados no Royal British Club em ...,3f51b5,NaN,2024-02-08T13:28:17Z,-1


In [36]:
columns_to_drop = ['Folder name', 'Phone number', 'Pin icon code']
incidents.drop(columns=columns_to_drop, inplace = True)

In [37]:
incidents.head(1)

,Folder color,Latitude,Longitude,Title,Description,Color,Timestamp
ID,,,,,,,
1,71b300,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,2024-02-08T17:16:00Z


In [38]:
incidents.isnull().sum()

Folder color    0
Latitude        0
Longitude       0
Title           0
Description     0
Color           0
Timestamp       0
dtype: int64

In [47]:
incidents['Color'].unique()

array(['71b300', '3f51b5', 'ff5722', 'ffeb3b', 'f44336', '9e9e9e',
       '795548', '9c27b0', '607d8b'], dtype=object)

In [40]:
incidents.head(3)

,Folder color,Latitude,Longitude,Title,Description,Color,Timestamp
ID,,,,,,,
1,71b300,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,2024-02-08T17:16:00Z
2,71b300,38.010202,-7.870127,1 de Fevereiro de 1985,"Bombas Detonadas, 8 dispositivos no Bairro Ale...",3f51b5,2024-02-08T17:37:17Z
3,71b300,38.748041,-9.188370,1 de Junho de 1984,"Agressão, disparos contra administrador de uma...",ff5722,2024-02-08T17:10:19Z


In [ ]:
incidents['Color'].replace('ff9800', 'ff5722', inplace=True)

In [48]:
incident_type = {'71b300' : 'Assalto', 
                '3f51b5': 'Bomba Detonadas',
                'ff5722' : 'Agressão',
                'ffeb3b' : 'Vandalismo',
                'f44336' : 'Assassinato',
                '9e9e9e' : 'Assalto com Vítimas Mortais', 
                '795548' : 'Tentativa de Assalto (falhada)', 
                '9c27b0' : 'Bombas Não Detonadas',
                '607d8b' : 'Fugas da Prisão'
                 }

incidents['Categoria'] = incidents['Color'].map(incident_type)

In [53]:
incidents

,Folder color,Latitude,Longitude,Title,Description,Color,Timestamp,Categoria
ID,,,,,,,,
1,71b300,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,2024-02-08T17:16:00Z,Assalto
2,71b300,38.010202,-7.870127,1 de Fevereiro de 1985,"Bombas Detonadas, 8 dispositivos no Bairro Ale...",3f51b5,2024-02-08T17:37:17Z,Bomba Detonadas
3,71b300,38.748041,-9.188370,1 de Junho de 1984,"Agressão, disparos contra administrador de uma...",ff5722,2024-02-08T17:10:19Z,Agressão
4,71b300,39.973942,-8.627772,1 de Maio de 1984,"Vandalismo, colocação de Pregos na Estrada Nac...",ffeb3b,2024-02-08T17:06:30Z,Vandalismo
5,71b300,38.815583,-9.223445,2 de Novembro de 1983,Assalto à dependência do Crédito Predial Portu...,71b300,2024-02-08T14:42:44Z,Assalto
...,...,...,...,...,...,...,...,...
119,71b300,38.768374,-9.294940,Janeiro de 1982,Bombas Detonadas no Posto da Guarda Nacional R...,3f51b5,2024-02-08T13:58:46Z,Bomba Detonadas
120,71b300,39.694550,-8.130265,Julho de 1980,Constante Vandalismo contra veiculos da Políci...,ffeb3b,2024-02-08T12:23:55Z,Vandalismo
121,71b300,38.697433,-9.423230,Maio de 1981,Explosivos Detonados no Royal British Club em ...,3f51b5,2024-02-08T13:28:17Z,Bomba Detonadas
